In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec  4 15:39 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [8]:
bucket_name = '316141407-bucket'

client = storage.Client()

# PageRank

In [20]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
  edges = pages.flatMap(lambda x: list(set([(x[0],i[0]) for i in x[1]])))
  vertices = edges.flatMap(lambda x: [(x[0],),(x[1],)])
  vertices = vertices.distinct()
  return edges, vertices

In [ ]:
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr_dict = {int(row.asDict()['id']) : row.asDict()['pagerank'] for row in pr.collect()}

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9913.728782160779|
|  10568| 5385.349263642039|
|  32927| 5282.081575765276|
|  30680| 5128.233709604123|
|5843419| 4957.567686263869|
|  68253| 4769.278265355159|
|  31717| 4486.350180548311|
|  11867|4146.4146509127695|
|  14533|3996.4664408855015|
| 645042|3531.6270898037437|
|  17867| 3246.098390604142|
|5042916| 2991.945739166178|
|4689264| 2982.324883041747|
|  14532| 2934.746829203171|
|  25391| 2903.546223513399|
|   5405|2891.4163291546347|
|4764461|2834.3669873326608|
|  15573| 2783.865118158838|
|   9316|2782.0396464137702|
|8569916|2775.2861918400163|
+-------+------------------+
only showing top 20 rows



In [ ]:
pr_file = 'pr.pkl'
with open(pr_file,'wb') as f:
    pickle.dump(pr_dict,f)

bucket = client.bucket(bucket_name)
blob = bucket.blob(f"pr/{pr_file}")
blob.upload_from_filename(pr_file)